# HSB Analysis

In [15]:
import glob
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
%matplotlib

Using matplotlib backend: Qt5Agg


## Giving the path

In [18]:
path = '/home/feli/LSBs_TFM/images/Datos/' # Path a los datos de perfiles

In [19]:
path_Ctes = '/home/feli/LSBs_TFM/listas/'

In [91]:
path_HSB_Data = '/home/feli/LSBs_TFM/Anlzds/TFM/HSB/'

## Reading the data

In [112]:
LSBs_list = sorted(glob.glob(path+'*.dat'))

In [113]:
Ctes_list = sorted(glob.glob(path_Ctes+'lista_run_a*'))

## This is the pixel scale and Freeman criteria

In [22]:
lsb_Freeman = 21.65 #Freeman et al. 1970
#NLSB = 21.0
#lsb_R = 23.3 #μ$_e$(R) $>$ 23.3 R mag arcsec$^{−2}$)
pscale = 0.396 #("/pix) from the SDSS data base

## Saving RA, DEC values

bucle = obj.tolist()
j = 0
len(bucle)
#len(Ctes_list)
#for j in range(Ctes_data):
#    print(obj[j])

i = 2
a = i# + 1 # numero de galaxias

Ctes_data = np.genfromtxt(Ctes_list[i], comments='#')[:,0:]


obj = Ctes_data[:, 0]
RA = Ctes_data[:, 2]
DEC = Ctes_data[:, 3]

while j < len(bucle):
    Coordenates = str(RA[j]) + '    ' + str(DEC[j]) + '    '
    HSBs_SDSS = open(path_Ctes+'RADEC_SDSS.dat', 'a') # Indicamos el valor 'w'.
    HSBs_SDSS.write(Coordenates+'\n')
    HSBs_SDSS.close()
    j = j + 1

print(obj, RA, DEC)#746 - 551, ' a las que sacar los datos',

## Coordenates

In [115]:
HSBs_data = path_Ctes+'HSB_SDSS.dat'
Datos_HSBs = np.genfromtxt(HSBs_data, comments='#')[:,0:]

In [122]:
obj = Datos_HSBs[:, 0]
RA = Datos_HSBs[:, 1]
DEC = Datos_HSBs[:, 2]
#'''
u = Datos_HSBs[:, 3]
g = Datos_HSBs[:, 4]
r = Datos_HSBs[:, 5]
i_color = Datos_HSBs[:, 6]
redshift = Datos_HSBs[:, 7]
#'''
print(obj[20], RA[20], DEC[20])#, u[232], g[232], r[232], i_color[232], redshift[232])

24.0 54.5509 1.16911


## Reading the data for the SB profile

In [130]:
i = 24
j = 20
a = i# + 1 # numero de galaxias

LSBs_raw_data = np.genfromtxt(LSBs_list[i], comments='#')[:,0:]

dat_name = LSBs_list[i][33:-4]#.astype(np.int16)
print(dat_name, a)#746 - 551, ' a las que sacar los datos',

024 24


In [131]:
sma = LSBs_raw_data[:, 0] * pscale #Semi-major axis
flux = LSBs_raw_data[:, 1]

## Calculating the limiting magnitude, colour and absolute magnitude

In [132]:
##CÁLCULO DE LA MAGNITUD LÍMITE

sigma = [4.842, 5.248, 6.349, 4.453, 5.434, 4.754, 4.507, 3.884, 6.407, 5.287]
zero_mag = 30.0

mu_lim = zero_mag - 2.5 * np.log10(3 * np.mean(sigma) / (pscale**2))
mag_lim = str(obj[j]) + '    ' + str(sigma) + '    ' + str(np.mean(sigma)) + '    ' + str(mu_lim)# + '    ' +

##CÁLCULO DE LOS COLORES

m_B = g[j] + 0.39*(g[j]-r[j]) + 0.21 #sigma 0.03
m_V = g[j] - 0.59*(g[j]-r[j]) - 0.01 #sigma 0.01
#g_r = 1.02*(m_B-m_V) - 0.220

##CÁLCULO DE LA MAGNITUD ABSOLUTA

#v = H_0 * D, v = z * c => z * c = H_0 *D => D = (z * c) / H_0
H_o = 67.3 # H0 de (67.3 ± 1.2) km/s/Mpc
c = 3e5 # km / s

d = ((redshift[j] * c) / H_o) * 1e6 # d tiene que ir en pc

M_abs = m_B + 5. - 5. * np.log10(d) # Calculo M(B), μ = m − M = 5logd − 5  ==> M = m + 5 - 5*log10 d

print(m_V, m_B, mu_lim, d, M_abs) 

15.7339 16.7281 25.0232403467 182763744.428 -19.5813502361


## Flux plot

In [133]:
# JESTER et al 2005 for stars

# Hago el cambio de r a g
#B_V = 0.62 * (g[j] - r[j]) + 0.15 # For quasars 
#B = g_SDSS + 0.17 * (u[j] - g[j]) + 0.11# For quasars

# FOR STARS WITH Rc - I_c < 1.15, AS INDICATE AT http://www.sdss3.org/dr8/algorithms/sdssUBVRITransform.php#Jester2005
B_V = 0.98 * (g[j]-r[j]) + 0.22 #sigma 0.04
g_SDSS = flux + 1.02 * B_V - 0.23

#g_SDSS = flux + 0.93 * B_V - 0.05 # flux es r, que es el filtro que tenemos

# Ahora calculo la magnitud Johnson en B
B = g_SDSS + 0.39 * (g[j] - r[j]) + 0.21 #sigma 0.03

'''
# Para el caso en que Rc_Ic > 1.15, usamos Lupton 2005

m_B = g[j] + 0.3130 * (g[j] - r[j]) + 0.2271
m_R = r[j] - 0.1837 * (g[j] - r[j]) - 0.097
m_V = g[j] - 0.5784*(g[j] - r[j]) - 0.0038

g_SDSS = m_B - m_R + flux - (0.33130 + 0.1837) * (g[j] - r[j]) + (0.2271 + 0.0971)

B = g_SDSS + 0.3130 * (g[j] - r[j]) + 0.2271
'''
# Calculo el brillo superficial convertido a Johnson
mu = 30.0 - 2.5 * np.log10(B) + 2.5 * np.log10(pscale**2)
#print(B - mu)

## Limit selection

In [134]:
fig,(ax1) = plt.subplots(nrows=1, sharex=True)
ax1.set_title('Este gráfico no lo guardo')
#ax0.errorbar(sma, flux, fmt='-o')
#ax0.set_ylabel('Flux (ADU)')

ax1.errorbar(sma, mu, fmt='-d')
plt.axhline(y=mu_lim, linewidth=1.5, color='r')
#plt.axvline(x=sma_break, linewidth=1.5, color='b')
#plt.plot(mu_lim, 'r-.', linewidth=1.5)
#ax1.set_ylim((26, 16))
ax1.set_xlabel('Galactocentric distance (arcsec)')
ax1.set_ylabel('Surface Brightness (mag/arcsec$^2$)')

ax1.invert_yaxis()

plt.show()

## Fitting the disk

### Outer disk fitting

In [142]:
rminodisk = 23.11
rmaxodisk = 31.97

m_odisk, b_odisk = np.polyfit(sma[(sma>rminodisk) & (sma<rmaxodisk)], mu[(sma>rminodisk) & (sma<rmaxodisk)], 1)
yr_outdisk = np.polyval([m_odisk, b_odisk], sma)
print('$\mu_0$ = ', b_odisk)
print('Pendiente = ', m_odisk)

$\mu_0$ =  22.7699285751
Pendiente =  0.0693875050228


## Inner disk fitting

In [143]:
rmin_indisk = 2.23
rmax_indisk = 23.10

m_indisk, b_indisk = np.polyfit(sma[(sma>rmin_indisk) & (sma<rmax_indisk)], mu[(sma>rmin_indisk) & (sma<rmax_indisk)], 1)
yr_indisk = np.polyval([m_indisk, b_indisk], sma)
print('$\mu_0$ = ', b_indisk)
print('Pendiente = ', m_indisk)

$\mu_0$ =  20.6729765923
Pendiente =  0.163200917097


## Break between fits

In [144]:
mu_break = ((m_indisk * b_odisk) - (m_odisk * b_indisk)) / (m_indisk - m_odisk)
#mu_break = b_odisk

# R_break
sma_break = (b_odisk - b_indisk) / (m_indisk - m_odisk)
#sma_break = 0.000
print(mu_break, sma_break)

24.3209036758 22.3523687755


# Graphics of the SB profile

## 2 plots

In [ ]:
fig,(ax0, ax1) = plt.subplots(nrows=2, sharex=True)
#ax0.set_title('%.3s, RA=%.3f, DEC=%.3f.' %(dat_name, RA, DEC))
ax0.errorbar(sma, flux, fmt='-o')
ax0.set_ylabel('Flux (ADU)')

#plt.title('Lin reg coef: m=%.3f, $\mu_{d, B}$=%.3f., $\mu_{b, B}$=%.3f. -- disk: $r_{min}$=%.3f, $r_{max}$=%.3f.'
#          %(md,bd,b,rmind,rmaxd))
ax1.errorbar(sma, mu, fmt='-d')
plt.plot(sma, yr_indisk, 'ro-', linewidth=1.5)
#plt.legend(['LSB disk'], loc = 1)
plt.plot(sma, yr_outdisk, 'go-', linewidth=1.5)

plt.axhline(y=mu_lim, linewidth=1.5, color='r', linestyle='--')

if b_indisk < lsb_Freeman and b_odisk > lsb_Freeman:
    plt.legend(['HSB disk', 'LSB disk'], loc = 1)
elif b_indisk > lsb_Freeman and b_odisk < lsb_Freeman:
    plt.legend(['LSB disk', 'HSB disk'], loc = 1)
elif b_indisk > lsb_Freeman and b_odisk > lsb_Freeman:
    plt.legend(['LSB disk', 'LSB disk'], loc = 1)    

'''
plt.plot(sma, yr1, 'b^-', linewidth=1.5)
plt.plot(sma, yr2, 'yv-', linewidth=1.5)
#plt.title('Lin reg coef: m=%.3f, $\mu_{d, B}$=%.3f., $\mu_{b, B}$=%.3f., $\mu_{1, B}$=%.3f., $\mu_{2, B}$=%.3f.'
#          %(m,bd,b,b1,b2))
'''
#ax1.set_ylim((26, 16))
ax1.set_xlabel('Galactocentric distance (arcsec)')
ax1.set_ylabel('Surf. Bright. (mag/arcsec$^2$)')

ax1.invert_yaxis()

multi_ajst = dat_name + '_4_Ajustes'

plt.savefig('/home/feli/LSBs_TFM/Anlzds/TFM/%.3s.png' %(dat_name))
#plt.savefig('/home/feli/LSBs_TFM/Anlzds/TFM/%.13s.png' %(multi_ajst))

plt.show()

## Only 1 plot

In [145]:
fig,(ax1) = plt.subplots(nrows=1, sharex=False)

#plt.title('Lin reg coef: m=%.3f, $\mu_{d, B}$=%.3f., $\mu_{b, B}$=%.3f. -- disk: $r_{min}$=%.3f, $r_{max}$=%.3f.'
#          %(md,bd,b,rmind,rmaxd))
ax1.errorbar(sma, mu, fmt='-d')
plt.plot(sma, yr_indisk, 'ro-', linewidth=1.5)
plt.plot(sma, yr_outdisk, 'go-', linewidth=1.5)
#plt.legend(['LSB disk'], loc = 1)

plt.axhline(y=mu_lim, linewidth=1., color='r', linestyle='--')
plt.errorbar(sma_break, mu_break, yerr=.5,  ecolor='black', linewidth=1., linestyle='--')

ax1.annotate('$R_{break}$',
            xy=(sma_break, mu_break), xycoords='data',
            xytext=(-80, -60), textcoords='offset points',
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc3,rad=.2"))


if b_indisk < lsb_Freeman and b_odisk > lsb_Freeman:
    plt.legend(['HSB disk', 'LSB disk'], loc = 1)
elif b_indisk > lsb_Freeman and b_odisk < lsb_Freeman:
    plt.legend(['LSB disk', 'HSB disk'], loc = 1)

#ax1.set_ylim((26, 16))
ax1.set_xlabel('Galactocentric distance (arcsec)')
ax1.set_ylabel('Surf. Bright. (mag/arcsec$^2$)')

ax1.invert_yaxis()

Big_name = dat_name + '_Ampliada'
multi_ajst = Big_name + '_4_Ajustes'

#plt.savefig('/home/feli/LSBs_TFM/Anlzds/TFM/%.12s.png' %(Big_name))
#plt.savefig('/home/feli/LSBs_TFM/Anlzds/TFM/%.18s.png' %(multi_ajst))

plt.show()

# Length scale

In [ ]:
#Cojo el disco externo porque quiero el disco entero
m_r, b_r = np.polyfit((sma[(sma>rmin_indisk) & (sma<rmaxodisk)]), np.log(mu[(sma>rmin_indisk) & (sma<rmaxodisk)]/m_odisk)**4, 1)
h_r_fit = np.polyval([m_r, b_r], 1./sma)
print('$\mu_0$ = ', b_r)
print('Pendiente = ', m_r)
h_r = b_r/1000.
print('h_r_OUT en kpc es:', h_r)

## Saving results and parameters

In [ ]:
argumento_write = str(obj[j]) + '    ' + str(RA[j]) + '    ' + str(DEC[j]) + '    ' + str(redshift[j]) + '    '
argumento_write = argumento_write + str(u[j]) + '    ' + str(g[j]) + '    ' + str(r[j]) + '    ' + str(i_color[j])

calculos_write = str(rminodisk) + '    ' + str(rmaxodisk) +  '    ' + str(b_odisk) + '    ' + str(m_odisk) + '    '
calculos_write = calculos_write + str(rmin_indisk) + '    ' + str(rmax_indisk) + '    ' + str(b_indisk) + '    ' + str(m_indisk) + '    '

magnitudes = str(mu_lim) + '    ' + str(B_V) + '    ' + str(m_B) + '    ' + str(m_V) + '    ' + str(b_indisk)
magnitudes = magnitudes + '    ' + str(b_odisk) + '    ' + str(M_abs) + '    ' + str(d) + '    ' + str(h_r) + '    ' + str(mu_break)

In [ ]:
tipoI = False
tipoII = False
tipoIII = False

In [ ]:
if tipoI is true:

### Saving IdObj, RA, DEC, u, g, r

    HSBs_dat_util = open(path_HSB_Data+'Type_I_HSB__SDSS.dat', 'a') # Indicamos el valor 'w'.
    HSBs_dat_util.write(argumento_write+'\n')
    HSBs_dat_util.close()

### Saving fitting parameters

    HSBs_calc = open(path_HSB_Data+'Type_I_HSB__param.dat', 'a') # Indicamos el valor 'w'.
    HSBs_calc.write(calculos_write+'\n')
    HSBs_calc.close()

### Saving magnitude calculus

    HSBs_mag = open(path_HSB_Data+'Type_I_HSB_mag.dat', 'a') # Indicamos el valor 'w'.
    HSBs_mag.write(magnitudes+'\n')
    HSBs_mag.close()

### Saving limiting magnitude data

    HBs_dat_util = open(path_HSB_Data+'Type_I_HSB_mag_lim.dat', 'a') # Indicamos el valor 'w'.
    HBs_dat_util.write(mag_lim+'\n')
    HBs_dat_util.close()
    
if tipoII is true:

### Saving IdObj, RA, DEC, u, g, r

    HSBs_dat_util = open(path_HSB_Data+'Type_II_HSB__SDSS.dat', 'a') # Indicamos el valor 'w'.
    HSBs_dat_util.write(argumento_write+'\n')
    HSBs_dat_util.close()

### Saving fitting parameters

    HSBs_calc = open(path_HSB_Data+'Type_II_HSB__param.dat', 'a') # Indicamos el valor 'w'.
    HSBs_calc.write(calculos_write+'\n')
    HSBs_calc.close()

### Saving magnitude calculus

    HSBs_mag = open(path_HSB_Data+'Type_II_HSB_mag.dat', 'a') # Indicamos el valor 'w'.
    HSBs_mag.write(magnitudes+'\n')
    HSBs_mag.close()

### Saving limiting magnitude data

    HBs_dat_util = open(path_HSB_Data+'Type_II_HSB_mag_lim.dat', 'a') # Indicamos el valor 'w'.
    HBs_dat_util.write(mag_lim+'\n')
    HBs_dat_util.close()

if tipoIII is true:

### Saving IdObj, RA, DEC, u, g, r

    HSBs_dat_util = open(path_HSB_Data+'Type_III_HSB__SDSS.dat', 'a') # Indicamos el valor 'w'.
    HSBs_dat_util.write(argumento_write+'\n')
    HSBs_dat_util.close()

### Saving fitting parameters

    HSBs_calc = open(path_HSB_Data+'Type_III_HSB__param.dat', 'a') # Indicamos el valor 'w'.
    HSBs_calc.write(calculos_write+'\n')
    HSBs_calc.close()

### Saving magnitude calculus

    HSBs_mag = open(path_HSB_Data+'Type_III_HSB_mag.dat', 'a') # Indicamos el valor 'w'.
    HSBs_mag.write(magnitudes+'\n')
    HSBs_mag.close()

### Saving limiting magnitude data

    HBs_dat_util = open(path_HSB_Data+'Type_III_HSB_mag_lim.dat', 'a') # Indicamos el valor 'w'.
    HBs_dat_util.write(mag_lim+'\n')
    HBs_dat_util.close()